# RWKV SageMaker LoRA Finetuning

RWKV を LoRA Finetuning し、SageMaker でデプロイするサンプルコード。

例として、日本語が 10% 学習に使用されている RWKV Raven 7B を利用します。必要に応じてモデルを変更してご利用ください。

In [ ]:
!pip install transformers

In [ ]:
import sagemaker, boto3, json, os
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

## Download and Tokenize Dataset

Fine Tuning 用の日本語データをフォルダに配置してアップロードします。

ここでは例として [Databricks Dolly 15k](https://github.com/databrickslabs/dolly/tree/master/data) データセットを日本語に翻訳したものを利用します。(License: [Creative Commons Attribution-ShareAlike 3.0 Unported License](https://creativecommons.org/licenses/by-sa/3.0/legalcode))

In [ ]:
# Download Databricks Dolly 15k dataset
!curl -L https://huggingface.co/datasets/kunishou/databricks-dolly-15k-ja/resolve/main/databricks-dolly-15k-ja.json --create-dirs -o ./data/databricks-dolly-15k-ja.json

In [ ]:
!head ./data/databricks-dolly-15k-ja.json

In [ ]:
# Download Tokenizer
!curl https://raw.githubusercontent.com/BlinkDL/RWKV-LM/main/RWKV-v4/20B_tokenizer.json --create-dirs -o data/20B_tokenizer.json

In [ ]:
# Tokenize Input
from transformers import PreTrainedTokenizerFast

def generate_prompt(instruction, input, output):
    if input != "":
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
# Instruction:
{instruction}
# Input:
{input}
# Response:
{output}
<|endoftext|>
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
# Instruction:
{instruction}
# Response:
{output}
<|endoftext|>
"""

def tokenize_and_save(input_file):
    dataset = pd.read_json(input_file)
    dataset = dataset.rename(columns={"context": "input", "response": "output"})
    dataset = dataset.to_dict(orient='records')

    output = ""
    for item in dataset:
        if len(item.keys()) > 0:
            output += generate_prompt(
                item["instruction"].strip(),
                item["input"].strip(),
                item["output"].strip()
            )

    np.set_printoptions(precision=4, suppress=True, linewidth=200)

    tokenizer = PreTrainedTokenizerFast(tokenizer_file='data/20B_tokenizer.json')
    output_file = 'data/train.npy'
    data_raw = output
    data_code = tokenizer.encode(data_raw)
    print(f'Tokenized length = {len(data_code)}')

    out = np.array(data_code, dtype='uint16')
    np.save(output_file, out, allow_pickle=False)

tokenize_and_save('data/databricks-dolly-15k-ja.json')

In [ ]:
input_train = sess.upload_data(
    path="./data/train.npy",
    key_prefix="Dolly"
)
input_train

## Prepare Model

In [ ]:
base_model_name = "rwkv-4-raven"
model_size = "3B"

# List of models trained on Japanese. Feel free to change here.
if model_size == "3B":
    base_model_file = "RWKV-4-Raven-3B-v11-Eng49%25-Chn49%25-Jpn1%25-Other1%25-20230429-ctx4096.pth"
    n_layer = 32
    n_embd = 2560
elif model_size == "7B":
    base_model_file = "RWKV-4-Raven-7B-v10-Eng89%25-Jpn10%25-Other1%25-20230420-ctx4096.pth"
    n_layer = 32
    n_embd = 4096

base_model_url = f"https://huggingface.co/BlinkDL/{base_model_name.lower()}/resolve/main/{base_model_file}"
base_model_path = f"scripts/code/base_models/{base_model_file}"

## Train

In [ ]:
hyperparameters={
    'load_model': f"/tmp/{base_model_file}",
    'model_url': base_model_url,
    'proj_dir': '/opt/ml/model',
    'data_file': '/opt/ml/input/data/train/train.npy',
    'data_type': 'numpy',
    'vocab_size': 50277,
    'ctx_len': 384,
    'epoch_save': 5,
    'epoch_count': 3,
    'max_epochs': 3, # Added to stop training
    'n_layer': n_layer,
    'n_embd': n_embd,
    'epoch_steps': 1000,
    'micro_bsz': 8,
    'accelerator': "gpu",
    'devices': 1,
    'precision': 'bf16',
    'lora': True,
    'lora_r': 8,
    'lora_alpha': 32,
    'lora_dropout': 0.01,
    'strategy': "deepspeed_stage_2",
    # 'enable_progress_bar': False, # Added to Reduce Log
}

In [ ]:
huggingface_estimator = HuggingFace(
    base_job_name="RWKV",
    role=role,
    entry_point='train.py',
    source_dir='./scripts/code',
    instance_type='ml.g5.2xlarge',
    instance_count=1,
    volume_size=200,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
    use_spot_instances=True,
    max_wait=86400,
    hyperparameters=hyperparameters,
)
huggingface_estimator.fit({'train': input_train})

## Download and Extract Model

In [ ]:
import boto3
import sagemaker

def get_latest_training_job_artifact(base_job_name):
    sagemaker_client = boto3.client('sagemaker')
    response = sagemaker_client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', SortOrder='Descending')
    training_job_arn = response['TrainingJobSummaries'][0]['TrainingJobArn']
    training_job_description = sagemaker_client.describe_training_job(TrainingJobName=training_job_arn.split('/')[-1])
    return training_job_description['ModelArtifacts']['S3ModelArtifacts']

try:
    model_data = huggingface_estimator.model_data
except:
    # Retrieve artifact url when kernel is restarted
    model_data = get_latest_training_job_artifact('RWKV')
    
!aws s3 cp {model_data} rwkv.tar.gz

In [ ]:
import tarfile
tarf = tarfile.open('rwkv.tar.gz', 'r:gz')
files = sorted(tarf.getnames())
target_file = files[-2]
print(files, target_file)

In [ ]:
!rm -rf scripts/model && mkdir scripts/model
!tar -xvf rwkv.tar.gz -C scripts/model --no-same-owner {target_file}
!cp data/20B_tokenizer.json scripts/model

## Package and Upload Model

In [ ]:
%cd scripts
!tar --exclude base_models -czvf ../package.tar.gz *
%cd -

In [ ]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"RWKV")
model_path

## Deploy Model

In [ ]:
from sagemaker.async_inference import AsyncInferenceConfig
from sagemaker.serializers import JSONSerializer

endpoint_name = "RWKV"

huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="1.13",
    py_version='py39',
    role=role,
    name=endpoint_name,
    env={
        "model_params": json.dumps({
            "model_url": base_model_url,
            "model_path": f"/tmp/{base_model_file}",
            "tokenizer_path": "model/20B_tokenizer.json", # path relative to model package
            "strategy": "cuda bf16",
            "lora": "model/" + target_file,
            "lora_alpha": hyperparameters["lora_alpha"]
        }),
    }
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    async_inference_config=AsyncInferenceConfig()
)

## Run Inference

In [ ]:
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.predictor_async import AsyncPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import NumpyDeserializer, JSONDeserializer

endpoint_name = "RWKV"

predictor_client = AsyncPredictor(
    predictor=Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker.Session(),
        serializer=JSONSerializer(),
        deserializer=NumpyDeserializer()
    ),
    name=endpoint_name
)
data = {
    "instruction": "ヴァージン・オーストラリア航空はいつから運航を開始したのですか？",
    "input": "ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・グループを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリア航空の破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。",
}
response = predictor_client.predict(
    data=data
)
print(response)

## Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()